<a href="https://colab.research.google.com/github/sonalshreya/LLM_Projects/blob/main/Simplifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate gradio bitsandbytes


In [ ]:
from huggingface_hub import login

login(token="hf-token")  # Paste your token here


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)


In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# model_id = "mistralai/Mistral-7B-Instruct-v0.3"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     load_in_8bit=True,
#     torch_dtype=torch.float16
# )


In [ ]:
def build_prompt(text):
    return f"""
Below is a technical passage. Generate three summaries:
1. An explanation for a 5-year-old.
2. A student-level explanation.
3. An expert-level summary.

Text:
{text}

Output:
"""


In [ ]:
def generate_summaries(text, max_tokens=512):
    prompt = build_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        do_sample=True
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("Output:")[-1].strip()


In [ ]:
import gradio as gr

def simplify(text):
    try:
        return generate_summaries(text)
    except Exception as e:
        return f"Error: {e}"

gr.Interface(
    fn=simplify,
    inputs=gr.Textbox(lines=10, label="Paste technical content here"),
    outputs=gr.Textbox(lines=20, label="Simplified Outputs"),
    title="🧠 ELI5 Text Simplifier",
    description="Get simplified summaries of complex text for kids, students, and experts."
).launch()
